In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam


In [ ]:
df = pd.read_csv("creditcard.csv")

# Use only "normal" (non-fraud) data for training
normal_data = df[df['Class'] == 0]
fraud_data  = df[df['Class'] == 1]

In [ ]:
# Drop 'Time' and 'Class'
X = normal_data.drop(['Time', 'Class'], axis=1)
X_test = df.drop(['Time', 'Class'], axis=1)
y_test = df['Class']

     

In [ ]:
# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)
     


In [ ]:
# Build Autoencoder 
input_dim = X_scaled.shape[1]

input_layer = Input(shape=(input_dim,))
encoded = Dense(16, activation='relu')(input_layer)
encoded = Dense(8, activation='relu')(encoded)

decoded = Dense(16, activation='relu')(encoded)
decoded = Dense(input_dim, activation='linear')(decoded)

autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')


In [ ]:
#   Train Autoencoder on Normal Data 
autoencoder.fit(X_scaled, X_scaled, epochs=10, batch_size=256, shuffle=True)

#  Predict and Calculate Reconstruction Error 
reconstructed = autoencoder.predict(X_test_scaled)
mse = np.mean(np.power(X_test_scaled - reconstructed, 2), axis=1)

     